<a href="https://colab.research.google.com/github/Luz-Aquino/chatbot-supermercado-ia/blob/main/Codigo_Supermercado_IA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gradio thefuzz groq

import sqlite3
import pandas as pd
from thefuzz import process
import gradio as gr
from groq import Groq
import re
from datetime import datetime

# --- 🔑 API KEY ---
API_KEY = "PEGÁ_TU_CLAVE_ACÁ"

try:
    client = Groq(api_key=API_KEY)
except:
    pass

# --- 1. BASE DE DATOS ---
conn = sqlite3.connect(':memory:', check_same_thread=False)
cursor = conn.cursor()

# Tablas
cursor.execute('DROP TABLE IF EXISTS usuarios')
cursor.execute('''CREATE TABLE usuarios (dni TEXT PRIMARY KEY, password TEXT, nombre TEXT, apellido TEXT, mail TEXT, fecha_nac TEXT, direccion TEXT)''')
cursor.execute('DROP TABLE IF EXISTS productos')
cursor.execute('''CREATE TABLE productos (id INTEGER PRIMARY KEY, nombre TEXT, marca TEXT, categoria TEXT, precio REAL, stock INTEGER, unidad_medida TEXT, promocion TEXT)''')

# Datos
datos_super = [
    (1, 'Leche Entera', 'La Serenísima', 'Lacteos', 1200.0, 50, '1 Litro', 'Llevando 3 pagas 2'),
    (2, 'Arroz Oro', 'Gallo', 'Almacen', 1850.5, 100, '1 kg', None),
    (3, 'Gaseosa Cola Original', 'Coca Cola', 'Bebidas', 2400.0, 40, '2.25 L', '2da al 50%'),
    (4, 'Fernet', 'Branca', 'Bebidas', 9500.0, 15, '750 ml', '10% OFF con App'),
    (5, 'Jabón Liquido', 'Ariel', 'Limpieza', 8200.0, 20, '3 L', None),
    (6, 'Yerba Mate', 'Playadito', 'Almacen', 3800.0, 60, '1 kg', 'Precio Cuidado'),
    (7, 'Fideos Tirabuzón', 'Matarazzo', 'Almacen', 1100.0, 80, '500 g', '3x2'),
    (8, 'Aceite Girasol', 'Natura', 'Almacen', 2100.0, 45, '1.5 L', None),
    (9, 'Azúcar Clásica', 'Ledesma', 'Almacen', 950.0, 90, '1 kg', None),
    (10, 'Harina 0000', 'Pureza', 'Almacen', 850.0, 70, '1 kg', None),
    (11, 'Puré de Tomate', 'Arcor', 'Almacen', 700.0, 100, '520 g', '4x3'),
    (12, 'Galletitas Rellenas', 'Oreo', 'Almacen', 1200.0, 150, '117 g', '2x1'),
    (13, 'Queso Cremoso', 'Cremón', 'Lacteos', 5500.0, 30, 'Por kg', None),
    (14, 'Manteca', 'La Paulina', 'Lacteos', 2200.0, 40, '200 g', None),
    (15, 'Yogur Bebible', 'Ser', 'Lacteos', 1500.0, 35, '1 kg', '20% OFF'),
    (16, 'Dulce de Leche', 'La Colonial', 'Almacen', 2800.0, 50, '400 g', None),
    (17, 'Cerveza Rubia', 'Quilmes', 'Bebidas', 1800.0, 120, '1 L', 'Pack x6 $9000'),
    (18, 'Vino Malbec', 'Rutini', 'Bebidas', 12500.0, 10, '750 ml', '15% OFF Caja'),
    (19, 'Agua Mineral', 'Villavicencio', 'Bebidas', 900.0, 200, '1.5 L', None),
    (20, 'Jugo Naranja', 'Citric', 'Bebidas', 2600.0, 25, '1 L', None),
    (21, 'Lavandina', 'Ayudin', 'Limpieza', 1200.0, 60, '1 L', None),
    (22, 'Detergente', 'Magistral', 'Limpieza', 2300.0, 50, '500 ml', '2x1'),
    (23, 'Papel Higiénico', 'Higienol', 'Limpieza', 3500.0, 40, '4 rollos', None),
    (24, 'Shampoo', 'Pantene', 'Perfumeria', 4200.0, 30, '400 ml', '25% OFF 2da unidad'),
    (25, 'Desodorante', 'Dove', 'Perfumeria', 2800.0, 45, '150 ml', None),
    (26, 'Pan Lactal', 'Artesano', 'Panaderia', 3200.0, 25, '500 g', None),
    (27, 'Huevos Blancos', 'Avicola', 'Frescos', 2800.0, 40, '12 un', 'Precio Amigo'),
    (28, 'Pechuga de Pollo', 'Granja Tres Arroyos', 'Carniceria', 6500.0, 20, 'Por kg', None),
    (29, 'Carne Picada', 'Frigorífico', 'Carniceria', 7200.0, 15, 'Por kg', None),
    (30, 'Manzanas Rojas', 'Moño Azul', 'Verduleria', 1800.0, 30, 'Por kg', 'Oferta de Estación')
]
cursor.executemany('INSERT INTO productos VALUES (?,?,?,?,?,?,?,?)', datos_super)
cursor.execute("INSERT INTO usuarios VALUES ('12345678', 'Luz12345', 'Luz', 'Aquino', 'luz@test.com', '1990-01-01', 'Calle Falsa 123')")
conn.commit()

# --- 2. VALIDACIONES ---
def es_mayor_de_edad(fecha_timestamp):
    if not fecha_timestamp: return False, "⚠️ Fecha obligatoria."
    try:
        fecha_nac = datetime.fromtimestamp(fecha_timestamp)
        hoy = datetime.now()
        if fecha_nac > hoy: return False, "⛔ Fecha futura inválida."
        edad = hoy.year - fecha_nac.year - ((hoy.month, hoy.day) < (fecha_nac.month, fecha_nac.day))
        if edad < 18: return False, f"⛔ Tenés {edad} años. Debés ser mayor de 18."
        return True, "OK"
    except: return False, "⚠️ Error en fecha."

def validar_formato_dni(dni):
    return bool(re.match(r'^\d{7,8}$', dni))

def validar_seguridad_password(password):
    if len(password) < 8: return False, "⚠️ Contraseña muy corta (min 8)."
    if not re.search(r"[A-Z]", password): return False, "⚠️ Falta una Mayúscula."
    if not re.search(r"[0-9]", password): return False, "⚠️ Falta un número."
    return True, "OK"

# --- 3. FUNCIONES DE USUARIO ---
def registrar_usuario(nombre, apellido, dni, mail, fecha, direccion, password):
    if not all([nombre, apellido, dni, mail, password]): return "⚠️ Faltan campos obligatorios (*)"
    if not validar_formato_dni(dni): return "⚠️ DNI inválido (Solo números)."
    es_mayor, msg_edad = es_mayor_de_edad(fecha)
    if not es_mayor: return msg_edad
    segura, msg_pass = validar_seguridad_password(password)
    if not segura: return msg_pass
    try:
        fecha_str = datetime.fromtimestamp(fecha).strftime('%Y-%m-%d')
        cursor.execute("INSERT INTO usuarios VALUES (?,?,?,?,?,?,?)", (dni, password, nombre, apellido, mail, fecha_str, direccion))
        conn.commit()
        return f"✅ ¡Usuario {nombre} registrado! Andá a Login."
    except sqlite3.IntegrityError: return "❌ DNI ya registrado."

def validar_login(dni, password):
    user = cursor.execute("SELECT * FROM usuarios WHERE dni=? AND password=?", (dni, password)).fetchone()
    if user:
        return f"👋 Hola {user[2]}", gr.update(visible=True), gr.update(visible=False)
    else:
        return "❌ Datos incorrectos", gr.update(visible=False), gr.update(visible=True)

def restaurar_password_real(dni, mail, nueva_password):
    user = cursor.execute("SELECT * FROM usuarios WHERE dni=? AND mail=?", (dni, mail)).fetchone()
    if not user: return "❌ Usuario no encontrado."
    segura, msg = validar_seguridad_password(nueva_password)
    if not segura: return msg
    cursor.execute("UPDATE usuarios SET password=? WHERE dni=?", (nueva_password, dni))
    conn.commit()
    return "✅ Contraseña cambiada con éxito."

def cerrar_sesion():
    return [gr.update(visible=False), gr.update(visible=True), "", "", "Sesión cerrada."]

# --- 4. RAG + IA ---
def buscar_datos(consulta):
    df = pd.read_sql_query("SELECT * FROM productos", conn)
    consulta = consulta.lower()
    if "bebida" in consulta: return "LISTA", df[df['categoria'] == 'Bebidas']
    elif "limpieza" in consulta: return "LISTA", df[df['categoria'] == 'Limpieza']
    elif "lacteo" in consulta: return "LISTA", df[df['categoria'] == 'Lacteos']
    elif "almacen" in consulta: return "LISTA", df[df['categoria'] == 'Almacen']
    elif "verdu" in consulta: return "LISTA", df[df['categoria'] == 'Verduleria']
    elif "carne" in consulta: return "LISTA", df[df['categoria'] == 'Carniceria']

    opciones = (df['marca'] + ' ' + df['nombre']).tolist()
    mejor, puntaje = process.extractOne(consulta, opciones)
    if puntaje > 55: return "PRODUCTO", df.iloc[opciones.index(mejor)]
    return "NADA", None

def responder_chat(mensaje):
    try:
        client = Groq(api_key=API_KEY)
        tipo, datos = buscar_datos(mensaje)
        if tipo == "LISTA":
            txt = datos[['nombre', 'marca', 'precio', 'promocion']].to_string(index=False)
            prompt = f"Usuario pregunta: '{mensaje}'. Tenemos: {txt}. Listalos con viñetas y resalta ofertas."
        elif tipo == "PRODUCTO":
            info = f"{datos['nombre']} ({datos['marca']}) ${datos['precio']}. Promo: {datos['promocion']}"
            prompt = f"Usuario busca: '{mensaje}'. Tenemos: {info}. Confirmale precio y oferta."
        else:
            prompt = f"Usuario busca '{mensaje}' y no tenemos. Pedí disculpas."

        completion = client.chat.completions.create(
            messages=[{"role": "user", "content": prompt}],
            model="llama-3.3-70b-versatile"
        )
        return completion.choices[0].message.content
    except Exception as e: return f"⚠️ Error IA: {str(e)}"

# --- 5. INTERFAZ GRÁFICA ---
with gr.Blocks(theme=gr.themes.Soft()) as app:
    gr.Markdown("# 🛒 Supermercado AI")

    with gr.Group() as bloque_acceso:
        with gr.Tab("🔐 Login"):
            dni_input = gr.Textbox(label="DNI *")
            pass_input = gr.Textbox(label="Contraseña *", type="password")
            login_btn = gr.Button("Ingresar", variant="primary")
            login_output = gr.Textbox(label="Estado", interactive=False)

        with gr.Tab("📝 Registro (+18)"):
            with gr.Row():
                reg_nombre = gr.Textbox(label="Nombre *")
                reg_apellido = gr.Textbox(label="Apellido *")
            with gr.Row():
                reg_dni = gr.Textbox(label="DNI *")
                reg_mail = gr.Textbox(label="Email *")
            reg_fecha = gr.DateTime(label="Fecha Nacimiento (+18) *", include_time=False)
            reg_dir = gr.Textbox(label="Dirección")
            reg_pass = gr.Textbox(label="Contraseña (Min 8, 1 Mayus, 1 Num) *", type="password")
            reg_btn = gr.Button("Crear Cuenta")
            reg_output = gr.Textbox(label="Resultado")

        with gr.Tab("🔄 Restaurar Clave"):
            res_dni = gr.Textbox(label="DNI")
            res_mail = gr.Textbox(label="Email")
            res_new_pass = gr.Textbox(label="Nueva Contraseña", type="password")
            res_btn = gr.Button("Cambiar")
            res_output = gr.Textbox(label="Estado")

    with gr.Group(visible=False) as bloque_chat:
        with gr.Row():
            gr.Markdown("## 💬 Chat de Compras")
            logout_btn = gr.Button("🚪 Salir", size="sm", variant="stop")

        chatbot_input = gr.Textbox(label="¿Qué buscás hoy?")
        chatbot_btn = gr.Button("Consultar")
        chatbot_output = gr.Markdown()

    # --- EVENTOS (ACÁ AGREGUÉ EL ENTER) ---

    # Login con Botón O con Enter en la contraseña
    login_btn.click(fn=validar_login, inputs=[dni_input, pass_input], outputs=[login_output, bloque_chat, bloque_acceso])
    pass_input.submit(fn=validar_login, inputs=[dni_input, pass_input], outputs=[login_output, bloque_chat, bloque_acceso])

    # Chat con Botón O con Enter
    chatbot_btn.click(fn=responder_chat, inputs=chatbot_input, outputs=chatbot_output)
    chatbot_input.submit(fn=responder_chat, inputs=chatbot_input, outputs=chatbot_output)

    # Demás botones
    reg_btn.click(fn=registrar_usuario, inputs=[reg_nombre, reg_apellido, reg_dni, reg_mail, reg_fecha, reg_dir, reg_pass], outputs=reg_output)
    res_btn.click(fn=restaurar_password_real, inputs=[res_dni, res_mail, res_new_pass], outputs=res_output)
    logout_btn.click(fn=cerrar_sesion, inputs=None, outputs=[bloque_chat, bloque_acceso, login_output, pass_input, login_output])

app.launch(share=True)